goal: predicts how many apples and oranges you can yield in a given condition

In [1]:
import numpy as np
import torch


In [2]:
#Sample dataset
# Input (temp, rainfall, humidity)
inputs = np.array([[73, 67, 43], 
                   [91, 88, 64], 
                   [87, 134, 58], 
                   [102, 43, 37], 
                   [69, 96, 70]], dtype='float32')
# Targets (apples, oranges)
targets = np.array([[56, 70], 
                    [81, 101], 
                    [119, 133], 
                    [22, 37], 
                    [103, 119]], dtype='float32')


# Convert inputs and targets to tensors
inputs = torch.from_numpy(inputs)
targets = torch.from_numpy(targets)


# Linear regression model from scratch

In [3]:
# Weights and biases
w = torch.randn(2, 3, requires_grad=True)
b = torch.randn(2, requires_grad=True)
def model(x):
    return x @ w.t() + b

# MSE loss
def mse(t1, t2):
    diff = t1 - t2
    return torch.sum(diff * diff) / diff.numel() #The .numel method of a tensor returns the number of elements in a tensor


In [4]:
#check loss before training
#check loss after training
print("true target:")
print(targets)
print('\n\nbefore training\n\n')
preds = model(inputs)
loss = mse(preds, targets)
print(preds)
print(loss)

# Train for 100 epochs
for i in range(100):
    preds = model(inputs)
    loss = mse(preds, targets)
    loss.backward() # update gradients for w and b
    with torch.no_grad():
        w -= w.grad * 1e-5
        b -= b.grad * 1e-5
        w.grad.zero_()
        b.grad.zero_()

#check loss after training
print('\n\nafter training\n\n')
preds = model(inputs)
loss = mse(preds, targets)
print(preds)
print(loss)

true target:
tensor([[ 56.,  70.],
        [ 81., 101.],
        [119., 133.],
        [ 22.,  37.],
        [103., 119.]])


 before training


tensor([[-63.0402, -20.5677],
        [-91.2943, -33.8550],
        [-44.4069, -48.6289],
        [-89.5197,   4.1957],
        [-81.2525, -49.8097]], grad_fn=<AddBackward0>)
tensor(20589.3633, grad_fn=<DivBackward0>)


 after training


tensor([[ 58.6936,  77.1255],
        [ 70.1989,  98.1681],
        [143.5416, 127.6912],
        [ 29.9326,  75.7014],
        [ 75.6054,  92.2531]], grad_fn=<AddBackward0>)
tensor(383.9767, grad_fn=<DivBackward0>)


# Linear regression built in

In [11]:
import torch.nn as nn
from torch.utils.data import TensorDataset
from torch.utils.data import DataLoader
import torch.nn.functional as F


In [6]:
# Input (temp, rainfall, humidity)
inputs = np.array([[73, 67, 43], 
                   [91, 88, 64], 
                   [87, 134, 58], 
                   [102, 43, 37], 
                   [69, 96, 70], 
                   [74, 66, 43], 
                   [91, 87, 65], 
                   [88, 134, 59], 
                   [101, 44, 37], 
                   [68, 96, 71], 
                   [73, 66, 44], 
                   [92, 87, 64], 
                   [87, 135, 57], 
                   [103, 43, 36], 
                   [68, 97, 70]], 
                  dtype='float32')

# Targets (apples, oranges)
targets = np.array([[56, 70], 
                    [81, 101], 
                    [119, 133], 
                    [22, 37], 
                    [103, 119],
                    [57, 69], 
                    [80, 102], 
                    [118, 132], 
                    [21, 38], 
                    [104, 118], 
                    [57, 69], 
                    [82, 100], 
                    [118, 134], 
                    [20, 38], 
                    [102, 120]], 
                   dtype='float32')

inputs = torch.from_numpy(inputs)
targets = torch.from_numpy(targets)



(tensor([73., 67., 43.]), tensor([56., 70.]))

In [8]:
train_ds = TensorDataset(inputs, targets)
train_ds[0]


(tensor([73., 67., 43.]), tensor([56., 70.]))

In [9]:
batch_size = 5
train_dl = DataLoader(train_ds, batch_size, shuffle=True)


In [12]:
model = nn.Linear(3, 2)
preds = model(inputs)
loss_fn = F.mse_loss
opt = torch.optim.SGD(model.parameters(), lr=1e-5)


In [13]:
def fit(num_epochs, model, loss_fn, opt, train_dl):
    
    # Repeat for given number of epochs
    for epoch in range(num_epochs):
        
        # Train with batches of data
        for xb,yb in train_dl:
            
            # 1. Generate predictions
            pred = model(xb)
            
            # 2. Calculate loss
            loss = loss_fn(pred, yb)
            
            # 3. Compute gradients
            loss.backward()
            
            # 4. Update parameters using gradients
            opt.step()
            
            # 5. Reset the gradients to zero
            opt.zero_grad()
        
        # Print the progress
        if (epoch+1) % 10 == 0:
            print('Epoch [{}/{}], Loss: {:.4f}'.format(epoch+1, num_epochs, loss.item()))


In [14]:
fit(100, model, loss_fn, opt, train_dl)

Epoch [10/100], Loss: 760.1420
Epoch [20/100], Loss: 292.8234
Epoch [30/100], Loss: 149.8422
Epoch [40/100], Loss: 153.1980
Epoch [50/100], Loss: 151.4576
Epoch [60/100], Loss: 34.7857
Epoch [70/100], Loss: 38.2837
Epoch [80/100], Loss: 48.2569
Epoch [90/100], Loss: 54.5574
Epoch [100/100], Loss: 16.6784
